<a href="https://colab.research.google.com/github/sushmitha6145/health-recommendation/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-telegram-bot --upgrade
!pip install Pillow
!pip install requests
!pip install opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [3]:
!pip install python-telegram-bot --upgrade


In [6]:
!pip uninstall python-telegram-bot -y
!pip install python-telegram-bot==13.7


Found existing installation: python-telegram-bot 21.2
Uninstalling python-telegram-bot-21.2:
  Successfully uninstalled python-telegram-bot-21.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3


In [2]:
!pip uninstall python-telegram-bot -y
!pip install python-telegram-bot==13.7


Found existing installation: python-telegram-bot 13.7
Uninstalling python-telegram-bot-13.7:
  Successfully uninstalled python-telegram-bot-13.7
  Using cached python_telegram_bot-13.7-py3-none-any.whl (490 kB)


In [1]:
import logging
from telegram import Update, Bot
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, ConversationHandler
import requests


In [ ]:
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, ConversationHandler

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Define states for conversation
ASK_PHOTO, ASK_DETAILS, PROVIDE_ADVICE = range(3)

def start(update: Update, context: CallbackContext) -> int:
    update.message.reply_text(
        "Hi! I'm your injury advisor bot. Please send me a photo of your injury."
    )
    return ASK_PHOTO

def photo(update: Update, context: CallbackContext) -> int:
    photo_file = update.message.photo[-1].get_file()
    context.user_data['photo'] = photo_file.download('injury.jpg')
    update.message.reply_text("Thanks! Now, can you tell me where the injury took place?")
    return ASK_DETAILS

def details(update: Update, context: CallbackContext) -> int:
    context.user_data['details'] = update.message.text
    update.message.reply_text("Got it. Can you describe the pain level (e.g., mild, moderate, severe)?")
    return PROVIDE_ADVICE

def advice(update: Update, context: CallbackContext) -> int:
    # Extract user data
    photo_path = context.user_data['photo']
    details = context.user_data['details']
    pain_level = update.message.text

    # Provide medical advice
    update.message.reply_text(f"Thank you for the information. Here is some advice:\n"
                               f"1. Clean the wound with antiseptic.\n"
                               f"2. Apply an antibiotic ointment.\n"
                               f"3. Take ibuprofen for pain relief.\n"
                               f"4. Avoid foods that can cause inflammation (e.g., junk food).\n"
                               f"5. Eat foods rich in vitamins and proteins.\n"
                               f"6. Don't worry, you will be fine!")

    # Reset user data
    context.user_data.clear()

    return ConversationHandler.END

def cancel(update: Update, context: CallbackContext) -> int:
    update.message.reply_text("Bye! Feel free to reach out if you need further assistance.")
    return ConversationHandler.END

def main() -> None:
    updater = Updater("6807966148:AAHGSAkpQZ4FZolQrK12qhkTi8_jr2etX48")

    dispatcher = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            ASK_PHOTO: [MessageHandler(Filters.photo, photo)],
            ASK_DETAILS: [MessageHandler(Filters.text & ~Filters.command, details)],
            PROVIDE_ADVICE: [MessageHandler(Filters.text & ~Filters.command, advice)]
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    dispatcher.add_handler(conv_handler)

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()
